# VerbaLex ASR Model

This notebook is intended to follow the tutorial (Fine tune Whisper model)[https://huggingface.co/blog/fine-tune-whisper] to make sure that I'm able to train a model on Kaggle. The model trained in this notebook is **not a great model** as it is only trained in 10 steps to facilitate debugging and testing.

# Prepare Environment


In [ ]:
!pip install --upgrade pip
!pip install --upgrade datasets accelerate soundfile librosa evaluate jiwer tensorboard gradio transformers

# Linking to Huggingface Hub
Linking this notebook to huggingface hub to enjoy these benefits:
* Integrated version control: you can be sure that no model checkpoint is lost during training.
* Tensorboard logs: track important metrics over the course of training.
* Model cards: document what a model does and its intended use cases.
* Community: an easy way to share and collaborate with the community!

This step is also **required** as we need authentication to use a dataset from Huggingface later to fine tune our base model.

To get the access token for login, head over to your Huggingface profile and press on`Settings`, then `Access Tokens`. Copy your tokens and paste it in the input box when prompted.

In [ ]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
login(token=user_secrets.get_secret("HF_TOKEN"))

# Load Dataset (Base model)
Loading dataset from Mozilla's Common voice dataset to develop a base ASR model.

Dataset link: https://huggingface.co/datasets/mozilla-foundation/common_voice_13_0

The dataset will need to be streamed from huggingface instead of downloading the whole thing. As it is too large for the Kaggle's notebook storage.

In [ ]:
from datasets import load_dataset, IterableDatasetDict

dataset = IterableDatasetDict()

dataset["train"] = load_dataset("mozilla-foundation/common_voice_13_0", "en", split="train", use_auth_token=True, streaming=True)
dataset["test"] = load_dataset("mozilla-foundation/common_voice_13_0", "en", split="test", use_auth_token=True, streaming=True)

Visualize to check that the data is indeed loaded

In [ ]:
print(list(dataset["train"].take(1)))
dataset

# Data Cleaning
As we can see from the data above, there are a lot of unnecessary data fields. We will drop those columns in the next cell by only selecting the columns that we need.

In [ ]:
dataset = dataset.select_columns(['audio', 'sentence'])
dataset

# Prepare Feature Extractor, Tokenizer and Data
The ASR pipeline can be decomposed into three components.
1. A feature extractor which pre-process the raw audio inputs. Learn more about it [here](https://huggingface.co/blog/fine-tune-whisper#load-whisperfeatureextractor).
2. The model which performs the sequence-to-sequence mapping. 
3. A tokenizer which post-process the model outputs to text format. Learn more about it [here](https://huggingface.co/blog/fine-tune-whisper#load-whispertokenizer).

The Whisper model has already provided a feature extractor, tokenizer as well as a processor. (Which combines the feature extractor and tokenizer, learn more about it [here](https://huggingface.co/blog/fine-tune-whisper#combine-to-create-a-whisperprocessor)) The following section will import those libraries and prepare the pipeline.

In [ ]:
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor

# Load feature extractor to process the raw audio inputs.
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

# Load Whisper tokenizer
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="English", task="transcribe")

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="English", task="transcribe")

## This Section is Optional
Optionally, we can verify that the tokenizer correctly encodes the characters by encoding and decoding a sample from the dataset.

### Decode function
There maybe a parameter within the `decode()` function that we are not familiar with, which is `skip_special_tokens`. Special tokens are added during encoding process. The tokenizer appends 'special tokens' at the start and the end of the sentence, including the start/end of the sentence, the language token, as well as any task tokens specified by `WhisperTokenizer.from_pretrained()` function above.

Hence, the `skip_special_tokens` parameter will specify whether do we want to include the 'special tokens' when we decode it into normal text.

In [ ]:
input_str = list(dataset["train"].take(1))[0]["sentence"]

# Encoding the sentence
labels = tokenizer(input_str).input_ids

# Decoding the sentence.
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

# Prepare data


Since the Whisper model was trained on data with a sampling rate of 16,000, we'll need to resample our data's sampling rate because the data provided by common voice has a sampling rate of 48,000.

In [ ]:
from datasets import Audio

dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))


list(dataset["train"].take(1))

After the sampling rate of our data has been resampled to 16,000kHz, we can then proceed to prepare our data for the model.

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]
    
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

dataset = dataset.map(prepare_dataset)
list(dataset["train"].take(1))

In [ ]:
dataset = dataset.remove_columns(["audio", "sentence"])
list(dataset["train"].take(1))

# Training and Evaluation

For training, the Huggingface Trainer will handle most of the work for us. All we have to do is:

* Define a data collator. A data collator will take our pre-processed data and prepares PyTorch tensors for the model.
* Evaluation metrics. We will need to define a `compute_metrics` function that computes the word error rate (WER) metric for evaluation.
* Load a pre-trained checkpoint and configure it correctly for training.
* Define training arguments for the Huggingface Trainer to construct a training schedule.

## Defining a Data Collator

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    
    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        
        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch
    
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

## Evaluation Metrics

In [ ]:
import evaluate

metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    
    label_ids[label_ids == -100] = tokenizer.pad_token_id
    
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    
    wer = 100 * metric.compute(predictions=pred_str, references=label_str)
    
    return {"wer": wer}

## Load a pre-trained checkpoint

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

model.generation_config.language = "<|en|>"
model.generation_config.task = "transcribe"

## Define training arguments

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-eng-gen",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=1,
    max_steps=10,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=5,
    eval_steps=5,
    logging_steps=5,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

## Training


In [ ]:
trainer.train() 